In [1]:
import os
from paddle.io import Dataset
import numpy as np
import paddle.nn.functional as F
import paddle as pp
import paddle
import pandas as pd
import paddle.nn as nn

In [2]:
sequence_length = 2500  #序列长度，最大帧为300，但这里还需要更改
input_size = 75       #输入数据特征大小 3（x,y,z）*25（关节数量）
hidden_size = 128     #隐藏层数据特征大小,即每个时间步对应的ht的维数
num_layers = 2        #隐藏层层数
num_classes = 30      #结果类数
batch_size = 20     #一个batch大小
num_epochs = 100       #epoch数目
learning_rate = 0.001  #学习率

In [3]:
class Reader(Dataset):
    def __init__(self, data_path, label_path, is_val: bool = False):
        super().__init__()
        self.data_list = np.load(data_path)
        self.label_list = np.load(label_path)
        self.index_list = list()
        for index in range(2922):
            if is_val and index % 5 == 1:
                self.index_list.append(index)
            elif not is_val and index % 5 != 1:
                self.index_list.append(index)

    def __getitem__(self, idx):
        idx = self.index_list[idx]
        x = self.data_list[idx, :, :, :, :]
        x = x.reshape(sequence_length,input_size)
        x = x.astype("float32")
        return x, self.label_list[idx]

    def __len__(self):
        return len(self.index_list)

In [4]:
data_path = '/Users/kongxuwen/Desktop/竞赛/滑冰/train_dataset/train_data.npy'
label_path = '/Users/kongxuwen/Desktop/竞赛/滑冰/train_dataset/train_label.npy'

In [5]:
a = Reader(data_path,label_path).__getitem__(1)
a[0].shape

(2500, 75)

In [6]:
class InferReader(Dataset):
    def __init__(self, INFER_DATA_PATH):
        super().__init__()
        self.data_list = np.load(INFER_DATA_PATH)
        self.index_list = list()
        for index in range(628):
            self.index_list.append(index)

    def __getitem__(self, idx):
        idx = self.index_list[idx]
        x = self.data_list[idx, :, :, :, :]
        x = x.transpose(1, 2, 0, 3)
        x = x.reshape((2500, 25, (1*3)))
        x = x.astype("float32")
        return x

    def __len__(self):
        return len(self.index_list)

In [7]:
class Net(pp.nn.Layer):
    def __init__(self, is_infer: bool = False):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        cell1 = nn.SimpleRNNCell(int(input_size/5), int(hidden_size/4))
        self.rnn = nn.RNN(cell1)
        cell2 = nn.SimpleRNNCell(hidden_size, hidden_size)
        self.rnn2 = nn.RNN(cell2)
        self.lstm = nn.LSTM(int(hidden_size/4*2*2), hidden_size, num_layers)
        
        self.fc = nn.Linear(hidden_size*sequence_length, num_classes)
        self.fs1 = nn.Linear(int(hidden_size/2),hidden_size)
        self.fs2 = nn.Linear(hidden_size*2,hidden_size)
    def forward(self, x):
        #layer1
        (x_p1,x_p2,x_p3,x_p4,x_p5) = paddle.chunk(x, 5,axis = 2)
        out1_p1,_ = self.rnn(x_p1)
        out1_p2,_ = self.rnn(x_p2)
        out1_p3,_ = self.rnn(x_p3)
        out1_p4,_ = self.rnn(x_p4)
        out1_p5,_ = self.rnn(x_p5)
        
        temp2_p1 = paddle.concat((out1_p1,out1_p2),2)
        temp2_p2 = paddle.concat((out1_p1,out1_p3),2)
        temp2_p3 = paddle.concat((out1_p1,out1_p4),2)
        temp2_p4 = paddle.concat((out1_p1,out1_p5),2)
        
        seqs = temp2_p1.shape[1]
        in2_p1 = []
        for seq in range(seqs):
            temp = paddle.squeeze(temp2_p1[:,seq,:],axis=1) #删除这个维度
            in2_p1_i = F.relu(self.fs1(temp))
            in2_p1.append(in2_p1_i)
        in2_p1 = paddle.stack(in2_p1,axis = 1)
        in2_p2 = []
        for seq in range(seqs):
            temp = paddle.squeeze(temp2_p2[:,seq,:],axis=1) #删除这个维度
            in2_p2_i = F.relu(self.fs1(temp))
            in2_p2.append(in2_p2_i)
        in2_p2 = paddle.stack(in2_p2,axis = 1)
        in2_p3 = []
        for seq in range(seqs):
            temp = paddle.squeeze(temp2_p3[:,seq,:],axis=1) #删除这个维度
            in2_p3_i = F.relu(self.fs1(temp))
            in2_p3.append(in2_p3_i)
        in2_p3 = paddle.stack(in2_p3,axis = 1)
        in2_p4 = []
        for seq in range(seqs):
            temp = paddle.squeeze(temp2_p4[:,seq,:],axis=1) #删除这个维度
            in2_p4_i = F.relu(self.fs1(temp))
            in2_p4.append(in2_p4_i)
        in2_p4 = paddle.stack(in2_p4,axis = 1)
        
        #layer2
        out2_p1,_ = self.rnn2(in2_p1)
        out2_p2,_ = self.rnn2(in2_p2)
        out2_p3,_ = self.rnn2(in2_p3)
        out2_p4,_ = self.rnn2(in2_p4)
        
        temp3_p1 = paddle.concat((out2_p1,out2_p2),2)
        temp3_p2 = paddle.concat((out2_p3,out2_p4),2)
        seqs = temp3_p1.shape[1]
        in3_p1 = []
        for seq in range(seqs):
            temp = paddle.squeeze(temp3_p1[:,seq,:],axis=1) #删除这个维度
            in3_p1_i = F.relu(self.fs2(temp))
            in3_p1.append(in3_p1_i)
        in3_p1 = paddle.stack(in3_p1,axis = 1)
        
        in3_p2 = []
        for seq in range(seqs):
            temp = paddle.squeeze(temp3_p2[:,seq,:],axis=1) #删除这个维度
            in3_p2_i = F.relu(self.fs2(temp))
            in3_p2.append(in3_p2_i)
        in3_p2 = paddle.stack(in3_p2,axis = 1)
        
        #layer3
        out3_p1,_ = self.rnn2(in3_p1)
        out3_p2,_ = self.rnn2(in3_p2)
        
        temp4_p1 = paddle.concat((out3_p1,out3_p2),2)
        in4_p1 = []
        for seq in range(seqs):
            temp = paddle.squeeze(temp4_p1[:,seq,:],axis=1) #删除这个维度
            in4_p1_i = F.relu(self.fs2(temp))
            in4_p1.append(in4_p1_i)
        in4_p1 = paddle.stack(in4_p1,1)
        
        #layer4
        out4,_ = self.lstm(in4_p1)
        out_ = self.fc(out4.reshape((out4.shape[0],hidden_size*sequence_length)))
        return out_

In [ ]:
input_define = pp.static.InputSpec(shape=[-1, 2500,75], dtype="float32", name='data')
label_define = pp.static.InputSpec(shape=[-1, 1], dtype="int64", name="label")
# 实例化网络对象并定义优化器等训练逻辑
model = pp.Model(Net(), inputs=input_define, labels=label_define)
# 此处lr使用CosineAnnealingDecay对学习率进行调整
lr = paddle.optimizer.lr.CosineAnnealingDecay(learning_rate=0.00625 , T_max=180, last_epoch=-1, verbose=False)
# 此处使用Momentum优化器
optimizer = pp.optimizer.Adam(learning_rate=lr, parameters=model.parameters())
# 损失函数使用交叉熵，评价指标使用准确率
# 其中Top-k表示推理得到的概率分布中，概率最高的前k个推理结果中是否包含正确标签，如果包含则视为正确，这里的1，2，3分别计算k为1~3的情况
model.prepare(optimizer=optimizer,
              loss=pp.nn.CrossEntropyLoss(),
              metrics=pp.metric.Accuracy(topk=(1, 5)))

model.fit(train_data=Reader(data_path=data_path,
                            label_path=label_path,
                            is_val=False),
          eval_data=Reader(data_path=data_path,
                           label_path=label_path,
                           is_val=True),
          batch_size=batch_size,
          epochs=num_epochs,
          save_dir="output/",
          save_freq=5,
          eval_freq=5,
          log_freq=20, 
          shuffle=True)
print("训练完毕")

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/100


/opt/anaconda3/envs/paddle/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  return (isinstance(seq, collections.Sequence) and
